# Average 3 Fold predictions

In [1]:
import json
import numpy as np
from pathlib import Path
import pandas as pd

# fold_paths = [
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-1",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-2",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-3"
# ]

fold_paths = [
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/RN50/Settings-combined/Fold-1",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/RN50/Settings-combined/Fold-1",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/RN50/Settings-combined/Fold-1"
]


# fold_paths = [
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-1",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-2",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-3"
# ]

# fold_paths = [
#     #"/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T123306_CellViT-SAM-L-Fold-1",
#     #"/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T123306_CellViT-SAM-L-Fold-2",
#     # "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T221050_CellViT-SAM-L-Fold-3"
# ]

In [2]:
detection_results = []
cell_pq_results = []
tissue_metrics = []
global_f1 = []
global_prec = []
global_rec = []


for fold, fold_path in enumerate(fold_paths):
    json_path = Path(fold_path) / "inference_results.json"
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    detection_results.append(pd.DataFrame.from_dict(data["nuclei_metrics_d"]))
    cell_pq_results.append(pd.DataFrame(data["nuclei_metrics_pq"], index=[0]))
    tissue_metrics.append(pd.DataFrame.from_dict(data["tissue_metrics"]))
    global_f1.append(float(data["dataset"]["f1_detection"]))
    global_prec.append(data["dataset"]["precision_detection"])
    global_rec.append(data["dataset"]["recall_detection"])

detection_avg = pd.concat([each.stack() for each in detection_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
cell_pq_avg = pd.concat([each.stack() for each in cell_pq_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()           
tissue_metrics_avg = pd.concat([each.stack() for each in tissue_metrics],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
             
global_f1 = np.mean(global_f1)
global_prec = np.mean(global_prec)
global_rec = np.mean(global_rec)

### Detection-Scores

In [3]:
detection_avg.T

,f1_cell,prec_cell,rec_cell
Neoplastic,0.666441,0.683828,0.649916
Inflammatory,0.565520,0.605344,0.530612
Connective,0.495989,0.499849,0.492189
Dead,0.023647,1.000000,0.011965
Epithelial,0.638029,0.704381,0.583101


In [4]:
print(f"f1: {global_f1}")
print(f"prec: {global_prec}")
print(f"rec: {global_rec}")

f1: 0.8187643746530257
prec: 0.8584138493007168
rec: 0.7826159467531877


### Cell-Type-PQ-Scores

In [5]:
cell_pq_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
0,0.542273,0.403763,0.369918,0.007775,0.507822


### Tissue-Type-PQ-Scores

In [6]:
tissue_metrics_avg.T

,Dice,Jaccard,mPQ,bPQ
adrenal_gland,0.813830,0.705797,0.475960,0.683147
bile-duct,0.768538,0.693955,0.434605,0.642038
bladder,0.754531,0.741326,0.525612,0.688671
breast,0.814806,0.705078,0.483402,0.653287
cervix,0.700083,0.723000,0.455550,0.664161
colon,0.679778,0.625657,0.399461,0.553335
esophagus,0.826642,0.719101,0.517089,0.655882
headneck,0.681020,0.651793,0.412501,0.609803
kidney,0.772801,0.719236,0.496914,0.679865
liver,0.845230,0.737427,0.457858,0.694340


In [7]:
row_mean = tissue_metrics_avg.mean(axis=1)
row_mean

Dice       0.790672
Jaccard    0.706005
mPQ        0.448240
bPQ        0.654142
dtype: float64

In [8]:
row_mean = tissue_metrics_avg.std(axis=1)
row_mean

Dice       0.052612
Jaccard    0.030016
mPQ        0.053745
bPQ        0.035015
dtype: float64